In [2]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
URL = "https://raw.githubusercontent.com/aviralb13/codes/main/datas/Churn_Modelling.csv"
data = pd.read_csv(URL)
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [104]:
x = data.iloc[:, 3:-1].values
y = data.iloc[:, -1].values


In [105]:
from sklearn.preprocessing import LabelEncoder
data = LabelEncoder()
x[:, 2] = data.fit_transform(x[:, 2])
print(x)


[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [106]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
column = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
x = np.array(column.fit_transform(x))
print(x)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [107]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y)

In [108]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [109]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
ann.fit(x_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
235/235 [==============================] - 3s 2ms/step - loss: 0.5248 - accuracy: 0.7957
Epoch 2/100
235/235 [==============================] - 1s 4ms/step - loss: 0.4661 - accuracy: 0.7957
Epoch 3/100
235/235 [==============================] - 1s 4ms/step - loss: 0.4474 - accuracy: 0.7957
Epoch 4/100
235/235 [==============================] - 2s 7ms/step - loss: 0.4389 - accuracy: 0.7957
Epoch 5/100
235/235 [==============================] - 1s 5ms/step - loss: 0.4336 - accuracy: 0.7957
Epoch 6/100
235/235 [==============================] - 1s 4ms/step - loss: 0.4293 - accuracy: 0.7957
Epoch 7/100
235/235 [==============================] - 0s 2ms/step - loss: 0.4258 - accuracy: 0.8031
Epoch 8/100
235/235 [==============================] - 1s 3ms/step - loss: 0.4229 - accuracy: 0.8092
Epoch 9/100
235/235 [==============================] - 1s 3ms/step - loss: 0.4200 - accuracy: 0.8141: 1s - loss: 0.4
Epoch 10/100
235/235 [==============================] - 1s 2ms/step - loss:

In [110]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

[[False]]


In [111]:
predictions = ann.predict(x_test)
predictions = (predictions > 0.5)

print(np.concatenate((predictions.reshape(len(predictions),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 0]
 [1 0]
 ...
 [0 0]
 [0 1]
 [0 0]]


In [112]:
from sklearn.metrics import  accuracy_score

accuracy_score(y_test, predictions)

0.8564